<br><font color = darkblue size=6><strong>Delta Hedge</strong></font> 

This is a Python translation from Richard's Excel on Delta Hedge

## Import libraries

In [1]:
# import libraries
import numpy as np
from scipy.stats import norm
import math
import pandas as pd
import cufflinks as cf
import warnings
warnings.filterwarnings("ignore")


## Calculate Asset Price (Excel cells B12-B112)

In [2]:
# define a function to calculate Asset Price
def asset_price(S0, Strike, IntRate, Vol, Maturity, timesteps):
    
    """S0: cell B1; Strike: cell B2; IntRate: cell B3; Vol: Cell B4; Maturity: Cell B5"""
    
    # Set the random seed for reproducibility
    # Same seed leads to the same set of random values
    np.random.seed(1) 
   
    # Define dt
    dt = Maturity/timesteps        # length of time interval  
    
    # Simulating asset prices with 't' timesteps
    S = np.zeros(timesteps+1)
    S[0] = S0

    for i in range(1, timesteps+1):
        z = np.random.standard_normal()     # psuedo random numbers
        S[i] = S[i-1] * np.exp((IntRate - 0.5 * Vol ** 2) * dt + Vol * np.sqrt(dt) * z) # vectorized operation per timesteps
        
    return S

In [3]:
# use user inputs in cells B1-B6 in Excel to calcuate cells B12-B112
# Note the outputs can be different from those in Excel due to randomization
asset = asset_price(100, 100, 0.05, 0.20, 1, 100)

## Calculate Delta_BS (Excel cells C12-C112)

In [4]:
# define function d1

def d1(asset, Vol, DivYld, IntRate, Strike, expiry):
    d1 = (np.log(asset / Strike) + (IntRate - DivYld + 0.5 * Vol * Vol) * expiry) / Vol / np.sqrt(expiry)
    return d1

# define function N1p
def N1p(asset, Vol, DivYld, IntRate, Strike, expiry):
    N1p = norm.cdf(d1(asset, Vol, DivYld, IntRate, Strike, expiry))
    return N1p

# define a function to calculate Delta_BS

def CallDelta(asset, Vol, DivYld, IntRate, Strike, expiry):
    CallDelta = np.exp(-DivYld * expiry) * N1p(asset, Vol, DivYld, IntRate, Strike, expiry)
    if asset[-1]>asset[0]:
        CallDelta[-1]=1
    else:
        CallDelta[-1]=0
    return CallDelta

In [5]:
# calculate cells B12-B112
# Note the outputs can be different from those in Excel due to randomization

Maturity = 1
Week = np.linspace(0,1,101) #Cells A12-A112
Delta_BS = CallDelta(asset, 0.20, 0, 0.05, 100, Maturity-Week )


## Calculate Bought/Sold (Excel cells D12-D112)

In [6]:
shares = 1 # cell B6

# shift Delta_BS by 1 and make the first element 0
Delta_BS_shift = np.roll(Delta_BS,1)
Delta_BS_shift[0] = 0

dDelta = (Delta_BS - Delta_BS_shift)*shares

## Calculate MtM (Excel cells E12-E112)

In [7]:
MtM = dDelta*asset

## Calculate Cum P&L (Excel cells F12-F112) and Interest (Excel Cells G12-G112)

In [8]:
# initiate Cum P&L
cum_pl = np.zeros(len(Week))
cum_pl[0] = MtM[0]

# initiate interest
IntRate = 0.05
interest = np.zeros(len(Week))
interest[0] = cum_pl[0]*(np.exp(IntRate*0.01)-1)

# calcualte Cum P&L and interest
for i in range (1, len(Week)):
    cum_pl[i] = MtM[i]+cum_pl[i-1]+interest[i-1]
    interest[i] = cum_pl[i]*(np.exp(IntRate*0.01)-1)
    
cum_pl # output cells F12-F112
interest # output cells G12-G112

array([0.03184949, 0.03492463, 0.03383227, 0.03286673, 0.03084664,
       0.03253917, 0.02815711, 0.03164485, 0.03015942, 0.03079776,
       0.03030289, 0.03327535, 0.02921575, 0.02855131, 0.02775374,
       0.03011323, 0.02784327, 0.02746699, 0.02562955, 0.02568683,
       0.02689501, 0.02456824, 0.02699624, 0.0289555 , 0.03005718,
       0.03206302, 0.03056433, 0.03028885, 0.02818596, 0.02756013,
       0.02876205, 0.02716006, 0.02621955, 0.02461364, 0.02266947,
       0.02113768, 0.02103826, 0.01862261, 0.01902023, 0.02267043,
       0.02437938, 0.02386139, 0.02170362, 0.01992168, 0.02392333,
       0.02398785, 0.02234623, 0.02274406, 0.02829204, 0.02859258,
       0.03027682, 0.03110657, 0.03012387, 0.02691447, 0.02588395,
       0.02523498, 0.0268751 , 0.02933628, 0.0321432 , 0.03302347,
       0.03572871, 0.03352423, 0.03740273, 0.03898884, 0.038231  ,
       0.03977027, 0.03967547, 0.04298383, 0.04676914, 0.05066943,
       0.04878541, 0.04609284, 0.04507769, 0.0456928 , 0.04794

## Calculate Gamma_BS (Excel cells I12-I112)

In [9]:
# define Nxd1
def Nxd1(asset, Vol, DivYld, IntRate, Strike, expiry):
    Nxd1 = 1 / np.sqrt(2 * math.pi) * np.exp(-0.5 * d1(asset, Vol, DivYld, IntRate, Strike, expiry)
                                             * d1(asset, Vol, DivYld, IntRate, Strike, expiry))
    return Nxd1
    


# define CallGamma function
def CallGamma(asset, Vol, DivYld, IntRate, Strike, expiry):
    CallGamma = (np.exp(-DivYld * expiry) * Nxd1(asset, Vol, DivYld, IntRate, Strike, expiry))/(Vol*asset*np.sqrt(expiry))
    CallGamma[-1] = 0
    return CallGamma


In [10]:
# initiate expiry
expiry = Maturity - Week

# calculate CallGamma (cells I12-I112)
Gamma_BS = CallGamma(asset, 0.2, 0 ,0.05, 100, expiry)

## Plot Cum P&L and Gamma_BS

In [11]:
# set up the dataframe
df = pd.DataFrame({'Cum P&L': cum_pl,
                   'Gamma_BS':Gamma_BS}, index=Week)

# plot
df.iplot(title='Cum P&L vs. Gamma_BS', secondary_y='Gamma_BS')